## Required Packages

In [ ]:
!pip install opencv-python
!pip install mediapipe

## Initial Setup

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

## Realtime Pose Estimation

In [ ]:
def find_angle_body_parts(first,second,third):
    first = np.array(first)
    second = np.array(second)
    third = np.array(third)

    radians = np.arctan2(third[1]-second[1], third[0]-second[0]) - np.arctan2(first[1]-second[1], first[0]-second[0])
    angle = np.abs(radians*180.0/np.pi)

    return 360 - angle if angle > 180.0 else angle

#### Test Pose Capture

In [ ]:
vidcap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while vidcap.isOpened():
        ret, frame = vidcap.read()

        # Return to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Run detection
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate angle
            angle = find_angle_body_parts(shoulder, elbow, wrist)

            # Visualize angle
            cv2.putText(image, str(angle),
                           tuple(np.multiply(elbow, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
        except:
            pass


        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Exercise Coach', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    vidcap.release()
    cv2.destroyAllWindows()

## Exercises

#### Planks

In [ ]:
vidcap = cv2.VideoCapture(0)
start_time = None
stage = 'Show'

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while vidcap.isOpened():
        ret, frame = vidcap.read()

        # Return to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Run detection
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # All body parts should be visible
            if landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility < 0.5 or \
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility < 0.5 or \
            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility < 0.5 or \
            landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility < 0.5:
                stage = 'Show full body'
                start_time = None
            else:
                # Get coordinates
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                # Calculate angle
                shoulder_hip_knee = find_angle_body_parts(shoulder, hip, knee)
                hip_knee_ankle = find_angle_body_parts(hip, knee, ankle)

                # Visualize angle
                cv2.putText(image, str(shoulder_hip_knee),
                            tuple(np.multiply(hip, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                cv2.putText(image, str(hip_knee_ankle),
                            tuple(np.multiply(knee, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                # Plank logic
                if shoulder_hip_knee > 160 and shoulder_hip_knee < 200 and \
                hip_knee_ankle > 160 and hip_knee_ankle < 200:
                    stage = "Good form"
                    if start_time is None:
                        start_time = time.time()
                else:
                    stage="Correct your form"
                    start_time = None
        except:
            pass

        # Setup status box
        cv2.rectangle(image, (0,0), (575,73), (245,117,16), -1)

        # Timer
        cv2.putText(image, 'TIME (s)', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(0 if start_time is None else int(time.time() - start_time)),
                    (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.7, (255,255,255), 2, cv2.LINE_AA)

        # Stage data
        cv2.putText(image, 'STAGE', (100,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage,
                    (95,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.7, (255,255,255), 2, cv2.LINE_AA)


        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Exercise Coach', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    vidcap.release()
    cv2.destroyAllWindows()

#### Lunges

In [ ]:
vidcap = cv2.VideoCapture(0)
counter = 0
stage = 'Show'

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while vidcap.isOpened():
        ret, frame = vidcap.read()

        # Return to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Run detection
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # All body parts should be visible
            if landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility < 0.5 or \
            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility < 0.5 or \
            landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility < 0.5 or \
            landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility < 0.5 or \
            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility < 0.5 or \
            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility < 0.5:
                stage = 'Show full body'
            else:
                if stage == 'Show full body':
                    stage = 'Begin' if counter == 0 else 'Keep going'
                # Get coordinates for left leg
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                # Get coordinates for right leg
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                # Calculate angle
                left_angle = find_angle_body_parts(left_hip, left_knee, left_ankle)
                right_angle = find_angle_body_parts(right_hip, right_knee, right_ankle)

                # Visualize angle
                cv2.putText(image, str(left_angle),
                            tuple(np.multiply(left_knee, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                cv2.putText(image, str(right_angle),
                            tuple(np.multiply(right_knee, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                # Lunge counter logic
                # There should be distance between the ankles (can't squat)
                if left_angle > 70 and left_angle < 110 and \
                right_angle > 70 and right_angle < 110 and \
                (left_ankle[0] - right_ankle[0])**2 + (left_ankle[1] - right_ankle[1])**2 > 0.1:
                    stage = "down"
                elif left_angle > 110 and right_angle > 110 and stage == 'down':
                    stage = "up"
                    counter += 1
                    print(counter)
        except:
            pass

        # Setup status box
        cv2.rectangle(image, (0,0), (575,73), (245,117,16), -1)

        # Rep data
        cv2.putText(image, 'REPS', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        # Stage data
        cv2.putText(image, 'STAGE', (100,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage,
                    (95,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.7, (255,255,255), 2, cv2.LINE_AA)


        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    vidcap.release()
    cv2.destroyAllWindows()

#### Shoulder Press

In [ ]:
vidcap = cv2.VideoCapture(0)
counter = 0
stage = 'Show'

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while vidcap.isOpened():
        ret, frame = vidcap.read()

        # Return to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Run detection
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # All body parts should be visible
            if landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility < 0.5 or \
            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility < 0.5 or \
            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility < 0.5 or \
            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility < 0.5 or \
            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility < 0.5 or \
            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility < 0.5 or \
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility < 0.5 or \
            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility < 0.5:
                stage = 'Show full body'
            else:
                if stage == 'Show full body':
                    stage = 'Begin' if counter == 0 else 'Keep going'

                # Get coordinates for left side
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                # Get coordinates for right side
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

                # Calculate angle
                left_elbow_angle = find_angle_body_parts(left_shoulder, left_elbow, left_wrist)
                right_elbow_angle = find_angle_body_parts(right_shoulder, right_elbow, right_wrist)
                left_shoulder_angle = find_angle_body_parts(left_hip, left_shoulder, left_elbow)
                right_shoulder_angle = find_angle_body_parts(right_hip, right_shoulder, right_elbow)

                # Visualize angle
                cv2.putText(image, str(left_elbow_angle),
                            tuple(np.multiply(left_elbow, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                cv2.putText(image, str(right_elbow_angle),
                            tuple(np.multiply(right_elbow, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                # Shoulder press counter logic
                # There should be distance between the elbow and hips (can't do bicep curls)
                if left_elbow_angle > 150 and right_elbow_angle > 150 and \
                left_shoulder_angle > 150 and right_shoulder_angle > 150:
                    stage = "up"
                elif left_shoulder_angle < 90 and right_shoulder_angle < 90 and stage == 'up':
                    stage = "down"
                    counter += 1
                    print(counter)
        except:
            pass

        # Setup status box
        cv2.rectangle(image, (0,0), (575,73), (245,117,16), -1)

        # Rep data
        cv2.putText(image, 'REPS', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        # Stage data
        cv2.putText(image, 'STAGE', (100,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage,
                    (95,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.7, (255,255,255), 2, cv2.LINE_AA)


        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Exercise Coach', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    vidcap.release()
    cv2.destroyAllWindows()